In [32]:
import openai
import os
import pandas as pd
import time
text_file = open("API_key.txt", "r")

folder_path = 'repo' # Replace with the path to your folder
file_paths_details = []

# Walk through all files and directories within the folder
for root, directories, files in os.walk(folder_path):
    for filename in files:
        # Append the path to each file to the file_paths list
        file_paths_details.append(os.path.join(root, filename))

# Print the list of file paths
print(file_paths_details)

#read whole file to a string
openai.api_key =  text_file.read()

#close file
text_file.close()


line_embeddings = []
    
def split_file_by_line(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        line_number = 0
        for i in lines:
            line_embeddings.append([filename,i,line_number])
            line_number +=1
    return 

blocks = []
def split_file_by_character(filename, character):
    with open(filename, 'r') as f:
        lines = f.readlines()

    start_line = 0
    prev_end_line = -1
    for i, line in enumerate(lines):
        if character in line:
            prev_end_line = i
            extracted_lines = lines[start_line:prev_end_line]

            # join the extracted lines into a string
            extracted_text = "".join(extracted_lines)
            if extracted_text != "":
                blocks.append([filename,start_line,prev_end_line,extracted_text])
            #print(filename)
            start_line = i 
    
    extracted_lines = lines[prev_end_line+1:len(lines)]
    # join the extracted lines into a string
    extracted_text = "".join(extracted_lines)
    blocks.append([filename,prev_end_line+1,len(lines),extracted_text])
    
    return 


comment_chars = {
    '.html':"<!-",
    '.css':"/*",
    '.js':"//"
}

for i in range(0,len(file_paths_details)):
        split_file_by_line(file_paths_details[i])
        file_extension = os.path.splitext(file_paths_details[i])[1]
        split_file_by_character(file_paths_details[i],comment_chars[file_extension])
        

print("Total number of functions extracted:", len(blocks))

['repo/index.html', 'repo/login-page.js', 'repo/style.css']
Total number of functions extracted: 10


In [33]:
df = pd.DataFrame(line_embeddings)
df2 = pd.DataFrame(blocks)
df.columns = ["filepath","Code","LineNumber"]
display(df)
df2.columns = ["filepath","BlockStart","BlockStop","Code"]
display(df2)

,filepath,Code,LineNumber
0,repo/index.html,<!DOCTYPE html>\n,0
1,repo/index.html,"<html lang=""en"">\n",1
2,repo/index.html,\n,2
3,repo/index.html,<!- File References ->\n,3
4,repo/index.html,\n,4
...,...,...,...
129,repo/style.css,font-weight: bold;\n,80
130,repo/style.css,background-color: #33c1ea;\n,81
131,repo/style.css,cursor: pointer;\n,82
132,repo/style.css,outline: none;\n,83


,filepath,BlockStart,BlockStop,Code
0,repo/index.html,0,3,"<!DOCTYPE html>\n<html lang=""en"">\n\n"
1,repo/index.html,3,14,<!- File References ->\n\n<head>\n <meta char...
2,repo/index.html,14,22,"<!- Login Page Holder ->\n<main id=""main-holde..."
3,repo/index.html,23,32,"<form id=""login-form"">\n <input type=""tex..."
4,repo/login-page.js,0,17,"\nconst loginForm = document.getElementById(""l..."
5,repo/style.css,0,4,/* Page Styles */\nhtml {\n height: 100%;\n}\n
6,repo/style.css,4,15,/* Body Styles */\nbody {\n height: 100%;\n ...
7,repo/style.css,15,27,/* Holder for login items*/\n#main-holder {\n ...
8,repo/style.css,27,53,/* Login Error Message Styles */\n#login-erro...
9,repo/style.css,54,85,#login-form {\n align-self: flex-start;\n di...


In [34]:
display(df)

,filepath,Code,LineNumber
0,repo/index.html,<!DOCTYPE html>\n,0
1,repo/index.html,"<html lang=""en"">\n",1
2,repo/index.html,\n,2
3,repo/index.html,<!- File References ->\n,3
4,repo/index.html,\n,4
...,...,...,...
129,repo/style.css,font-weight: bold;\n,80
130,repo/style.css,background-color: #33c1ea;\n,81
131,repo/style.css,cursor: pointer;\n,82
132,repo/style.css,outline: none;\n,83


In [10]:
df['code_embedding'] = ''
def get_embedding(task):
    time.sleep(2.5)
    response = openai.Embedding.create(
            input=task,
            model="text-embedding-ada-002"
        )
    return response['data'][0]['embedding']
i=0
for ind in df.index:
        i+=1
        df['code_embedding'][ind] = get_embedding(df['Code'][ind])
        print(f"Counting down: i", end="\r")
print("Done")
df.to_csv("df.csv", index=False)


1


/var/folders/tb/txlhvbmn2fgct223r9bxzyc80000gn/T/ipykernel_70872/2093251899.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['code_embedding'][ind] = get_embedding(df['Code'][ind])


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
Done


In [ ]:
df = pd.read_csv('df.csv')
df['code_embedding'] = df.code_embedding.apply(lambda x: [float(y) for y in x[1:-1].split(",")])
display(df)

In [11]:
display(df)

,filepath,Code,LineNumber,code_embedding
0,repo/index.html,<!DOCTYPE html>\n,0,"[-0.01256885752081871, 0.0028874874114990234, ..."
1,repo/index.html,"<html lang=""en"">\n",1,"[-0.009006879292428493, 0.010268648155033588, ..."
2,repo/index.html,\n,2,"[0.0025078936014324427, -0.018215738236904144,..."
3,repo/index.html,<!- File References ->\n,3,"[-0.025017328560352325, -0.0020156055688858032..."
4,repo/index.html,\n,4,"[0.0025078936014324427, -0.018215738236904144,..."
...,...,...,...,...
129,repo/login-page.js,location.reload();\n,12,"[-0.018045900389552116, -0.0164205189794302, -..."
130,repo/login-page.js,} else {\n,13,"[-0.03089103475213051, -0.012493962422013283, ..."
131,repo/login-page.js,loginErrorMsg.style.opacity = 1;\n,14,"[-0.007993947714567184, -0.014100635424256325,..."
132,repo/login-page.js,}\n,15,"[-0.019705967977643013, -0.017968423664569855,..."


In [12]:
import numpy 
import matplotlib
task = 'Add this image in the login page holder'
from openai.embeddings_utils import cosine_similarity

def search_functions(df, code_query):
    embedding = get_embedding(code_query)
    df['similarities'] = df.code_embedding.apply(lambda x: cosine_similarity(x, embedding))
    res = df.sort_values('similarities', ascending=False).head(100)
    return res

res = search_functions(df, task)

135


In [20]:
df3 = df.sort_values('similarities', ascending=False).head(round(0.05*len(df)))
display(df3)
print(len(df3))

,filepath,Code,LineNumber,code_embedding,similarities
14,repo/index.html,<!- Login Page Holder ->\n,14,"[-0.03042474575340748, 0.023209165781736374, -...",0.855454
8,repo/index.html,<title>Login</title>\n,8,"[-0.0014111034106463194, 0.011160314083099365,...",0.818872
60,repo/style.css,#login-error-msg-holder {\n,28,"[-0.03985173627734184, 0.01832166686654091, 0....",0.817035
47,repo/style.css,/* Holder for login items*/\n,15,"[-0.013486423529684544, 0.01937653310596943, 0...",0.807583
17,repo/index.html,"<h1 id=""login-header"">Login</h1>\n",17,"[0.0010761611629277468, 0.025847628712654114, ...",0.799559
26,repo/index.html,"<input type=""submit"" value=""Login"" id=""log...",26,"[-0.020360980182886124, -0.0023479533847421408...",0.796043
86,repo/style.css,#login-form {\n,54,"[-0.004600471816956997, 0.01568753831088543, 0...",0.795649


7


In [24]:
df3 = pd.DataFrame()

In [18]:
df2["Hits"] = 0
display(df2)

,filepath,BlockStart,BlockStop,Code,Hits
0,repo/index.html,0,3,"<!DOCTYPE html>\n<html lang=""en"">\n\n",0
1,repo/index.html,3,14,<!- File References ->\n\n<head>\n <meta char...,0
2,repo/index.html,14,22,"<!- Login Page Holder ->\n<main id=""main-holde...",0
3,repo/index.html,23,32,"<form id=""login-form"">\n <input type=""tex...",0
4,repo/style.css,0,4,/* Page Styles */\nhtml {\n height: 100%;\n}\n,0
5,repo/style.css,4,15,/* Body Styles */\nbody {\n height: 100%;\n ...,0
6,repo/style.css,15,27,/* Holder for login items*/\n#main-holder {\n ...,0
7,repo/style.css,27,53,/* Login Error Message Styles */\n#login-erro...,0
8,repo/style.css,54,85,#login-form {\n align-self: flex-start;\n di...,0
9,repo/login-page.js,1,17,"const loginForm = document.getElementById(""log...",0


In [26]:
# group by filepath and concatenate LineNumber
df3 = res.groupby("filepath").agg({"LineNumber": list}).reset_index()

display(df3)


,filepath,LineNumber
0,repo/index.html,"[14, 8, 17, 26]"
1,repo/style.css,"[28, 15, 54]"


In [28]:
import numpy as np
def count_lines(filepath, start, stop):
    count = 0
    for i in df3[df3['filepath']==filepath]['LineNumber']:
        #print(i)
        for j in i:
            if(j>=start and j<stop):
                count +=1
    return count

# apply the function to each row of df2 and create a new column
df2['Hits'] = df2.apply(lambda row: count_lines(row['filepath'], row['BlockStart'], row['BlockStop']), axis=1)
display(df2)
df2.to_csv("df2.csv", index=False)


,filepath,BlockStart,BlockStop,Code,Hits
0,repo/index.html,0,3,"<!DOCTYPE html>\n<html lang=""en"">\n\n",0
1,repo/index.html,3,14,<!- File References ->\n\n<head>\n <meta char...,1
2,repo/index.html,14,22,"<!- Login Page Holder ->\n<main id=""main-holde...",2
3,repo/index.html,23,32,"<form id=""login-form"">\n <input type=""tex...",1
4,repo/style.css,0,4,/* Page Styles */\nhtml {\n height: 100%;\n}\n,0
5,repo/style.css,4,15,/* Body Styles */\nbody {\n height: 100%;\n ...,0
6,repo/style.css,15,27,/* Holder for login items*/\n#main-holder {\n ...,1
7,repo/style.css,27,53,/* Login Error Message Styles */\n#login-erro...,1
8,repo/style.css,54,85,#login-form {\n align-self: flex-start;\n di...,1
9,repo/login-page.js,1,17,"const loginForm = document.getElementById(""log...",0


In [29]:
display(df2)
df2.to_csv("df2.csv", index=False)


,filepath,BlockStart,BlockStop,Code,Hits
0,repo/index.html,0,3,"<!DOCTYPE html>\n<html lang=""en"">\n\n",0
1,repo/index.html,3,14,<!- File References ->\n\n<head>\n <meta char...,1
2,repo/index.html,14,22,"<!- Login Page Holder ->\n<main id=""main-holde...",2
3,repo/index.html,23,32,"<form id=""login-form"">\n <input type=""tex...",1
4,repo/style.css,0,4,/* Page Styles */\nhtml {\n height: 100%;\n}\n,0
5,repo/style.css,4,15,/* Body Styles */\nbody {\n height: 100%;\n ...,0
6,repo/style.css,15,27,/* Holder for login items*/\n#main-holder {\n ...,1
7,repo/style.css,27,53,/* Login Error Message Styles */\n#login-erro...,1
8,repo/style.css,54,85,#login-form {\n align-self: flex-start;\n di...,1
9,repo/login-page.js,1,17,"const loginForm = document.getElementById(""log...",0


In [30]:
df2 = df2.sort_values('Hits', ascending=False)   
print(df2.iloc[0])

filepath                                        repo/index.html
BlockStart                                                   14
BlockStop                                                    22
Code          <!- Login Page Holder ->\n<main id="main-holde...
Hits                                                          2
Name: 2, dtype: object
